In [ ]:
import alpaca_trade_api as alpaca
from dotenv import load_dotenv
import os
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import time

In [ ]:
# initialize API from API keys in .env
load_dotenv()
api_key = os.environ['APCA-API-KEY-ID']
api_secret_key = os.environ['APCA-API-SECRET-KEY']
api_base_url = 'https://paper-api.alpaca.markets'
api = alpaca.REST(api_key, api_secret_key, api_base_url)
trading_client = TradingClient(api_key, api_secret_key, paper=True)

In [ ]:
account = api.get_account()
print(account)

In [ ]:
def get_SP500():
    return pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [ ]:
def market_buy_order(ticker, quantity):
    time.sleep(1)
    market_order_data = MarketOrderRequest(
                        symbol=ticker,
                        qty=quantity,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.DAY
                        )
    market_order = trading_client.submit_order(order_data=market_order_data)
    print(market_order)

In [ ]:
def market_sell_order(ticker, quantity):
    time.sleep(1)
    market_order_data = MarketOrderRequest(
                        symbol=ticker,
                        qty=quantity,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.DAY
                        )
    market_order = trading_client.submit_order(order_data=market_order_data)
    print(market_order)

In [ ]:
SP500 = get_SP500()
SP500

In [ ]:
now = dt.datetime.now(dt.timezone.utc)
todays_date = now.strftime("%Y-%m-%d")
market_open_datetime = todays_date + "T14:40:00Z"
current_datetime = now.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
if now.hour >= 21:
    current_datetime = todays_date + "T20:50:00Z"
print(todays_date)
print(market_open_datetime)
print(current_datetime)
today_SPY_bars = api.get_bars("JPM", alpaca.TimeFrame(1, alpaca.TimeFrameUnit.Minute), market_open_datetime, current_datetime, feed='iex').df

today_SPY_bars

In [ ]:
close_min = today_SPY_bars['close'].min()
support_threshold = 0.001
lowest_bars = today_SPY_bars[today_SPY_bars['close'] <= close_min * (1 + support_threshold)]
print(close_min)
lowest_bars

In [ ]:
# for i in range(5):
#     market_buy_order('SPY', 1)
#     market_sell_order('SPY', 1)

In [ ]:
# check if the current price is near a support level from at least 30 minutes ago

In [ ]:
print(today_SPY_bars.tail(1).index)
today_SPY_bars.tail(1)

In [ ]:
support_threshold = 0.001 # 0.1%

for ticker in SP500['Symbol']:
    time.sleep(0.05)
    bars = api.get_bars(ticker, alpaca.TimeFrame(1, alpaca.TimeFrameUnit.Minute), market_open_datetime, current_datetime, feed='iex').df
    close_min = bars['close'].min()
    support_line = close_min * (1 + support_threshold)
    if bars.tail(1).iloc[0]['close'] <= support_line:
        print(ticker)
    # lowest_bars = today_SPY_bars[today_SPY_bars['close'] <= close_min * (1 + support_threshold)]
    # print(close_min)
    # lowest_bars